In [12]:
import os
import autogen
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from autogen import register_function
from typing import Annotated, Literal

In [13]:
# def parse_files_in_directory(directory_path):
    
#     parsed_data = parse_pdfs(directory_path)

#     # for filename in os.listdir(directory_path):
#         # file_path = os.path.join(directory_path, filename)
        
#         # if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
#         #     loader = UnstructuredImageLoader(file_path)
#         #     documents = loader.load()
#         #     parsed_text = ' '.join([doc.page_content for doc in documents])

#         # else:
#         #     continue

#         # parsed_data[filename] = parsed_text

#     return parsed_data


def parse_pdfs(directory:str ="../data/") -> dict:
# def parse_pdfs(dir_path="../data/"):
    parsed_data = {}
    loader = PyPDFDirectoryLoader(directory, extract_images=True)
    documents = loader.load()
    for page in documents:
        if parsed_data.get(page.metadata["source"]) is None:
            parsed_data[page.metadata["source"]] = [page.page_content]
        else:
            parsed_data[page.metadata["source"]].append(page.page_content)
    return parsed_data


In [14]:
api_key = "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"

config_list = [
  {
    "model": "llama-3.2-3b-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
    "api_rate_limit": 100
  }
]

# llm_config = config_list[0]
llm_config={"config_list": config_list}

In [15]:
task = "Parse the documents and classify each of them by a heading"

In [16]:
# user_proxy = autogen.ConversableAgent(
#     name="Admin",
#     system_message="Give the task, and send instructions to doc_classifier to classify the documents in the data folder",
#     code_execution_config=False,
#     llm_config=llm_config,
#     human_input_mode="ALWAYS",
# )

user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=10,
    # code_execution_config={
    #     "work_dir": "code",
    #     "use_docker": False
    # },
    llm_config=False,
)

In [17]:
doc_parse_caller = autogen.ConversableAgent(
    name="Doc Parse Caller",
    llm_config=llm_config,
    description="A caller for parsing documents",
    system_message="You are a helpful AI assistant. Use the provided python function parse_pdfs(directory), to parse the documents at the directory '../data/'."
)

doc_parser = autogen.ConversableAgent(
    name="Doc Parser",
    llm_config=llm_config,
    # system_message="You are a helpful AI assistant as an executer that parses documents. If you receive a python call instruction, then you execute it, or else ignore the message."
)

doc_classifier = autogen.ConversableAgent(
    name="Doc Classifier",
    llm_config=llm_config,
    description="A classifier that classifies the text that has been parsed",
    system_message="You are a helpful AI assistant."
                   "For each document provided, categorize them from one of these options ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other']."
                   "Reply TERMINATE when the task is done.",
)



In [18]:
# user_proxy.register_function(
#     function_map={
#         "parse_pdfs": parse_pdfs()
#     }
# )

# register_function(
#     parse_pdfs,
#     caller=doc_parse_caller,  
#     executor=doc_parser,  
#     name="parse_pdfs",  
#     description="A simple document parser",  # A description of the tool.
# )

# Register the tool signature with the assistant agent.
doc_parse_caller.register_for_llm(name="parse_pdfs", description="A simple pdf parser")(parse_pdfs)

# Register the tool function with the user proxy agent.
doc_parser.register_for_execution(name="parse_pdfs")(parse_pdfs)

<function __main__.parse_pdfs(directory: str = '../data/') -> dict>

In [19]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, doc_parser, doc_classifier],
    messages=[],
    max_round=5,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [doc_parser],
        doc_parser: [doc_classifier],
        doc_classifier: [],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [20]:
doc_parse_caller.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple pdf parser',
   'name': 'parse_pdfs',
   'parameters': {'type': 'object',
    'properties': {'directory': {'type': 'string',
      'default': '../data/',
      'description': 'directory'}},
    'required': []}}}]

In [21]:
def parse_calling(recipient, messages, sender, config):
    # return f"Can you provide the python function to call for parsing the documents?\n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"
    #print(f"{recipient.chat_messages_for_summary(sender)[-1]['content']}")
    return "I have some pdf documents in the directory '../data/', and I would like to parse the documents in this directory"


doc_parser.register_nested_chats(
    [
        {
            "recipient": doc_parse_caller,
            "message": parse_calling,
            "max_turns": 2,
            "summary_method": "last_msg",
        }
    ],
    trigger=manager,  # condition=my_condition,
)

In [22]:
result = user_proxy.initiate_chat(
    manager,
    # message="Parse the documents and classify each of them by a heading",
    message="I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)",
)

User (to chat_manager):

I have some pdf documents in the directory '../data/', and I would like to know what type of documents they are (in terms of classification)

--------------------------------------------------------------------------------

Next speaker: Doc Parser


>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
Doc Parser (to Doc Parse Caller):

I have some pdf documents in the directory '../data/', and I would like to parse the documents in this directory

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


Doc Parse Caller (to Doc Parser):

***** Suggested tool call (call_pp94): parse_pdfs *****
Arguments: 
{"directory": "../data/"}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION parse_pdfs...
Doc Parser (to Doc Parse Caller):

Doc Parser (to Doc Parse Caller):

***** Response from calling tool (call_pp94) *****
{"../data/Belgium RP.pdf": ["5133640 12\nTREDE SEJOUR\nRESIDENCE PERMIT\nN REGISTRE NATI0NAL 95.07.30-528.34\nREMARQUESIREMARKS\nA.SEJOUR LIMITE\nCAT.DU TITRE / TYPE OF PERMIT\nF/F\nSEX\nSEXEI\nKoumudiYasara-G\nGANEPOLA ACHCHIGE\nNOM Prenoms/SURNAME.Forenames\nLKA\nNATIONALITY\nNATIONALITE\n513364012\nVALABLE JUSQU'AU /CARD EXPIRY\n31 10 2023\n30071995\nDATE DE NAISSANCE\nDATE OF BIRTH", "12102022\nATEDEDELIVRANCE\nETUDIANT\nMARCHEDUTRAVAILLIMITE\nREMARQUES/REMARKS\nATEOFISSUE\nGANEPOLAKACHCHIGE<<KOUMUDIKYAS\n9507306F2310314LKA950730

/home/koumudi/Documents/Llama_Hack/LlamaHackers/venv/lib/python3.12/site-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.2-3b-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)



Next speaker: User

User (to chat_manager):



--------------------------------------------------------------------------------
